In [5]:
"""
Project: ACSE Supermarket Data Analysis - Data Understanding
Authors: Jasmine Xu, Amelie Ndongozi, Jiyuan Xin, Miao Yin, Yifeng Dong
Date:    March 8 2025
"""

'\nProject: ACSE Supermarket Data Analysis - Data Understanding\nAuthors: Jasmine Xu, Amelie Ndongozi, Jiyuan Xin, Miao Yin, Yifeng Dong\nDate:    March 8 2025\n'

# 1. Load Data

In [6]:
import os
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
con = duckdb.connect(database=':memory:') 

In [8]:
#duckdb.execute("COPY (SELECT * FROM read_csv('ACSE Data/products.csv', parallel=True)) TO 'ACSE Data/products.parquet' (FORMAT 'parquet');")
products_path = 'ACSE Data/products.parquet'

In [9]:
# duckdb.execute("COPY (SELECT * FROM read_csv('ACSE Data/transactions.csv', parallel=True)) TO 'ACSE Data/transactions.parquet' (FORMAT 'parquet');")
trans_path = 'ACSE Data/transactions.parquet'

### 1.1 Understand the Store Distribution

In [10]:
query_store_distribution = """
WITH store_customer_counts AS (
  SELECT 
    store_id, 
    COUNT(DISTINCT cust_id) AS total_store_customers
  FROM read_parquet('ACSE Data/transactions.parquet')
  GROUP BY store_id
)

SELECT 
    total_store_customers,
    COUNT(*) AS num_stores,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () AS percentage
FROM store_customer_counts
GROUP BY total_store_customers
ORDER BY num_stores DESC;
"""
df_store_distribution = duckdb.query(query_store_distribution).to_df()
print(df_store_distribution)


IOException: IO Error: No files found that match the pattern "ACSE Data/transactions.parquet"

#Smallest store has 305 customers.
Largest store has 780,607 customers.
Many stores have 200,000+ customers.

### 1.2 Understand Customer Distribution 

In [ ]:
query_distribution = """
WITH customer_spending AS (
    SELECT 
        cust_id, 
        SUM(sales_amt) AS total_spent,
        CASE 
            WHEN SUM(sales_amt) < 0 THEN 'negative_spender'
            WHEN SUM(sales_amt) < 50 THEN 'low_spender'
            WHEN SUM(sales_amt) < 100 THEN 'mid_spender'
            WHEN SUM(sales_amt) < 1000 THEN 'high_spender'
            ELSE 'vip'
        END AS spending_segment
    FROM read_parquet('ACSE Data/transactions.parquet')
    GROUP BY cust_id
)

SELECT 
    spending_segment,
    COUNT(*) AS num_customers,
    COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () AS percentage
FROM customer_spending
GROUP BY spending_segment
ORDER BY num_customers DESC;
"""
df_distribution = duckdb.query(query_distribution).to_df()

print(df_distribution)


#low_spenders (42.1%) and high_spenders (30.9%) dominate the dataset, while negative_spenders (0.47%) and VIPs (11.5%) are underrepresented

In [ ]:
# Query to get the date range and count unique stores
query = """
SELECT 
    MIN(trans_dt) AS start_date,
    MAX(trans_dt) AS end_date,
    COUNT(DISTINCT store_id) AS unique_stores
FROM read_parquet('ACSE Data/transactions.parquet') """
df_summary = con.execute(query).fetchdf()
print(df_summary)


##  1.3 Sample Transactions Table

In [ ]:
query_trans = """
WITH customer_segments AS (
  SELECT 
    store_id,
    cust_id,
    SUM(sales_amt) AS total_sales,
    CASE 
      WHEN SUM(sales_amt) < 0 THEN 'negative_spender'
      WHEN SUM(sales_amt) < 50 THEN 'low_spender'
      WHEN SUM(sales_amt) < 100 THEN 'mid_spender'
      WHEN SUM(sales_amt) < 1000 THEN 'high_spender'
      ELSE 'vip'
    END AS spending_segment
  FROM read_parquet('ACSE Data/transactions.parquet')
  GROUP BY store_id, cust_id
),

sampled_customers AS (
  SELECT * 
  FROM customer_segments 
  WHERE RANDOM() <= 0.05
)

SELECT t.*
FROM read_parquet('ACSE Data/transactions.parquet') t
JOIN sampled_customers cs
ON t.store_id = cs.store_id AND t.cust_id = cs.cust_id;

"""
df_transactions = con.execute(query_trans).fetchdf()
print("Unique stores sampled:", df_transactions['store_id'].nunique())
print(df_transactions.shape)  
print(df_transactions.head())


## 1.4 Inspect the sampled data 

Q1: If all store has been sampled? 

In [ ]:
# Print number of unique stores in the sampled data
print("Unique stores sampled:", df_transactions['store_id'].nunique())

Q2: If the Date Range matches the original dataset? 

In [ ]:
#Check Date Range
start_date = df_transactions['trans_dt'].min()
end_date = df_transactions['trans_dt'].max()

print(f"Sampled Transactions Date Range: {start_date} to {end_date}")

Q3: If the distribution is the same as the original dataset? 

In [ ]:
# Check Customer Spending Segment Distribution
customer_distribution = df_transactions.groupby('cust_id')['sales_amt'].sum().reset_index()
customer_distribution['spending_segment'] = pd.cut(
    customer_distribution['sales_amt'],
    bins=[-float('inf'), 0, 50, 100, 1000, float('inf')],
    labels=['negative_spender', 'low_spender', 'mid_spender', 'high_spender', 'vip']
)

df_customer_dist = customer_distribution['spending_segment'].value_counts(normalize=True) * 100
print("\nCustomer Spending Distribution (as % of sampled data):")
print(df_customer_dist)

In [ ]:
# Check Store-Level Customer Distribution
store_customer_counts = df_transactions.groupby('store_id')['cust_id'].nunique().reset_index()
store_customer_counts.columns = ['store_id', 'total_store_customers']

df_store_dist = store_customer_counts['total_store_customers'].describe()
print("\nStore Customer Distribution Summary:")
print(df_store_dist)

## 1.5 Load Products Table

In [ ]:
df_products = duckdb.query(f"SELECT * FROM read_parquet('{products_path}')").fetchdf()
print(df_products.head())

In [ ]:
# Register the Pandas DataFrame as a DuckDB table named "df_transactions" and 'df_products'
con.register('df_transactions', df_transactions)
con.register('df_products', df_products)

# 2. EDA

## 2.1 Quick Data Check

In [ ]:
# Print data types for df_transactions
print("Data types for df_transactions:")
print(df_transactions.dtypes)

print("\nDetailed info for df_transactions:")
print(df_transactions.info())

In [ ]:
# Print data types for df_products
print("\nData types for df_product:")
print(df_products.dtypes)
print("\nDetailed info for df_products:")
print(df_products.info())

In [ ]:
# Basic transactions table overview
print("New Sampled transactions table Data:")
display(df_transactions.head())

In [ ]:
# Basic products table overview
print("New Sampled products table Data:")
display(df_products.head())

## 2.2 Check Null Value and Duplicates

In [ ]:
#Check Missing Values
print("Missing Values in Trans Table")
print(df_transactions.isnull().sum())
print("Missing Values in Products Table")
print(df_products.isnull().sum())

In [ ]:
# Check for full duplicate rows in the products table
duplicates_products = df_products[df_products.duplicated()]
num_duplicates_products = duplicates_products.shape[0]
print(f"Total duplicate rows in the products table: {num_duplicates_products}")

In [ ]:
# Check for full duplicate rows in the transactions table
duplicates_trans = df_transactions[df_transactions.duplicated()]
num_duplicates_trans = duplicates_trans.shape[0]
print(f"Total duplicate rows in the transactions table: {num_duplicates_trans}")

In [ ]:
# Remove Full Duplicates in Products
df_products = df_products.drop_duplicates(keep='first')

In [ ]:
# Remove Full Duplicates in transactions
df_transactions = df_transactions.drop_duplicates(keep='first')

In [ ]:
print(df_products.isnull().sum())

In [ ]:
## Inspecting missing value
import missingno as msno
msno.matrix(df_products, figsize=(5,5))
plt.title("Missing Data Pattern - Productws")
plt.show()

In [ ]:
# Drop rows where all values are null
df_products_cleaned = df_products.dropna(how='all')

# Fill missing values in prod_type with 'Unknown'
df_products_cleaned['prod_type'].fillna('Unknown', inplace=True)

# Display summary of missing values after cleaning
print("Missing values after cleaning:")
print(df_products_cleaned.isnull().sum())

## 2.3 Check Product Categories 

In [ ]:
# Perform a left join on prod_id to combine transaction details with product details
df_combined = df_transactions.merge(
    df_products,  
    on='prod_id', 
    how='left'
)

print("Merged Transactions and Products Table:")
display(df_combined.head())
print(df_combined.info())

In [ ]:
top_product_per_category = df_combined.loc[
    df_combined.groupby('prod_category')['sales_amt'].idxmax(),
    ['prod_category', 'prod_id', 'sales_amt']
]

category_sales_summary = df_combined.groupby('prod_category').agg(
    total_sales_amt=('sales_amt', 'sum'), 
    total_sales_count=('sales_qty', 'sum') 
).reset_index()


category_sales_summary = category_sales_summary.merge(top_product_per_category, on='prod_category', how='left')
category_sales_summary = category_sales_summary.sort_values(by='total_sales_amt', ascending=False)

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

print("Sales Amount and Sales Count by Product category with Top Product:")
display(category_sales_summary)


In [ ]:
query_category_subcategory = f"""
SELECT 
    prod_category,
    prod_subcategory,
    SUM(sales_amt) AS sales_amt
FROM df_combined
GROUP BY prod_category, prod_subcategory
ORDER BY prod_category, sales_amt DESC
"""

df_category_subcategory = con.execute(query_category_subcategory).fetchdf()
display(df_category_subcategory)


In [ ]:
# Drop the Scanning errors , prod_id is 20093960 for all tables 
df_transactions = df_transactions[df_transactions['prod_id'] != 20093960]
df_products = df_products[df_products['prod_id'] != 20093960]
df_combined = df_combined[df_combined['prod_id'] != 20093960]
# Display confirmation
print("Removed prod_id 20093960 from all datasets.")


In [ ]:
# Register the Pandas DataFrame as a DuckDB table named "df_combined"
con.register('df_combined',df_combined)

# Q1 Who are the best customers in terms of revenues, profits, transactions/store visits, number of products, etc.?

In [ ]:
query_best_customers = """
        SELECT 
        t.cust_id, 
        COUNT(DISTINCT t.trans_id) AS total_transactions,
        COUNT(DISTINCT t.store_id) AS total_store_visits,
        COUNT(DISTINCT t.prod_id) AS total_products_purchased,
        SUM(t.sales_amt) AS total_revenue
    FROM df_combined t
    GROUP BY t.cust_id
"""

df_analysis = con.execute(query_best_customers).fetchdf()

In [ ]:
#best customers in terms of revenues
df_revenue = df_analysis.groupby("cust_id")["total_revenue"].sum().reset_index()

df_revenue = df_revenue.sort_values(by="total_revenue", ascending=False)

print(df_revenue.head(20).to_string(index=False))


In [ ]:
#best customers in terms of transactions
df_transactions = df_analysis.groupby("cust_id")["total_transactions"].sum().reset_index()

df_transactions = df_transactions.sort_values(by="total_transactions", ascending=False)

print(df_transactions.head(20).to_string(index=False))


In [ ]:
#best customers in terms of store visits
df_visits = df_analysis.groupby("cust_id")["total_store_visits"].sum().reset_index()

df_visits = df_visits.sort_values(by="total_store_visits", ascending=False)

print(df_visits.head(20).to_string(index=False))

In [ ]:
#best customers in terms of number of products
df_products_purchased = df_analysis.groupby("cust_id")["total_products_purchased"].sum().reset_index()

df_products_purchased = df_products_purchased.sort_values(by="total_products_purchased", ascending=False)

print(df_products_purchased.head(20).to_string(index=False))

In [ ]:
best_100_customers = df_revenue.head(100)['cust_id']

df_top100 = df_combined[df_combined['cust_id'].isin(best_100_customers)]

df_top100['trans_dt'] = pd.to_datetime(df_top100['trans_dt'])

df_top100['weekday'] = df_top100['trans_dt'].dt.day_name()

df_top100_weekday = df_top100.groupby('weekday').size().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)
print(df_top100_weekday)


In [ ]:
import matplotlib.pyplot as plt

df_combined['trans_dt'] = pd.to_datetime(df_combined['trans_dt'])

df_combined['weekday'] = df_combined['trans_dt'].dt.day_name()

df_all_customers_weekday = df_combined.groupby('weekday').size().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)


import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
df_all_customers_weekday.plot(kind='bar')

plt.title('Overall Customer Transactions by week')
plt.ylabel('Number of Transactions')

plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')
plt.tight_layout()

plt.show()

In [ ]:
transaction_products = df_combined.groupby('trans_id')['prod_id'].nunique().reset_index(name='num_products')

transaction_type_counts = transaction_products['num_products'].apply(
    lambda x: 'Bulk Purchase' if x > 1 else 'Single Purchase'
).value_counts()

transaction_type_counts.plot(
    kind='pie',
    autopct='%1.1f%%',
    title='Bulk Purchase vs Single Purchase Transactions',
    ylabel='',
    figsize=(6, 6)
)

plt.axis('equal')
plt.show()

In [ ]:
from itertools import combinations
from collections import Counter

transactions_products = df_combined.groupby('trans_id')['prod_desc'].apply(set)

combination_counter = Counter()
for products in transactions_products:
    if len(products) >= 2:
        combination_counter.update(combinations(sorted(products), 2))

common_combinations = pd.DataFrame(
    combination_counter.most_common(10), columns=['Product Combination', 'Count']
)

comb_labels = [' + '.join(combo) for combo in combination_counter.keys()][:10]
comb_counts = [count for count in combination_counter.values()][:10]

plt.barh(comb_labels[::-1], comb_counts[::-1])

plt.title('Top 10 Frequent Product Combinations')
plt.xlabel('Number of Transactions')
plt.ylabel('Product Combinations')

plt.tight_layout()

plt.show()

In [ ]:
df_combined['cust_id_str'] = df_combined['cust_id'].astype(str)
df_combined['is_rewards_member'] = df_combined['cust_id_str'].apply(
    lambda x: 'Member' if x.startswith('1') else 'Non-Member'
)

In [ ]:
rewards_summary = df_combined.groupby('is_rewards_member').agg(
    total_revenue=('sales_amt', 'sum'), 
    total_transactions=('trans_id', 'nunique'), 
    avg_revenue_per_trans=('sales_amt', 'mean'), 
).reset_index()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

sns.set_style('whitegrid')

plt.figure(figsize=(8, 5))
barplot = sns.barplot(
    data=rewards_summary, 
    x='is_rewards_member', 
    y='total_transactions', 
    palette='Blues'
)

for patch in barplot.patches:
    height = patch.get_height()
    x_pos = patch.get_x() + patch.get_width() / 2
    barplot.text(
        x_pos, height, 
        f'{height:,.0f}', 
        ha='center', va='bottom', 
        fontsize=10
    )

plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

plt.title('Total Transactions: Member vs Non-Member', fontsize=14, fontweight='bold')
plt.xlabel('Customer Type', fontsize=12)
plt.ylabel('Total Transactions', fontsize=12)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
barplot = sns.barplot(
    data=rewards_summary, 
    x='is_rewards_member', 
    y='total_revenue', 
    palette='Blues'
)

for patch in barplot.patches:
    height = patch.get_height()
    x_pos = patch.get_x() + patch.get_width() / 2  
    barplot.text(
        x_pos, height,
        f'{height:,.2f}', 
        ha='center', va='bottom', 
        fontsize=10
    )

plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

plt.title('Total Revenue: Member vs Non-Member')
plt.xlabel('Customer Type')
plt.ylabel('Total Revenue')

plt.tight_layout()
plt.show()

# Q2. What are the products and product groups with the best volumes, revenues, profits, transactions, customers, etc.?

In [ ]:
query_analysis = """
        SELECT 
        prod_id, 
        prod_desc, 
        prod_category, 
        prod_subcategory, 
        COUNT(DISTINCT trans_id) AS total_transactions, 
        COUNT(DISTINCT cust_id) AS unique_customers,
        SUM(sales_qty) AS total_sales_volume,
        SUM(sales_amt) AS total_revenue
    FROM df_combined
    GROUP BY prod_id, prod_desc, prod_category, prod_subcategory

"""

df_analysis = con.execute(query_analysis).fetchdf()


### Display By Revenues
df_category_revenue = df_analysis.groupby("prod_category")["total_revenue"].sum().reset_index()

df_category_revenue = df_category_revenue.sort_values(by="total_revenue", ascending=False)

plt.figure(figsize=(18, 6))

sns.barplot(data=df_category_revenue, x="prod_category", y="total_revenue", palette="viridis")

plt.title("Total Revenue Distribution by Product Category", fontsize=14)
plt.xlabel("Product Category", fontsize=12)
plt.ylabel("Total Revenue", fontsize=6)
plt.xticks(rotation=45, ha="right") 
plt.show()



In [ ]:
### Display By Total Volume
df_category_sales = df_analysis.groupby("prod_category")["total_sales_volume"].sum().reset_index()
df_category_sales = df_category_sales.sort_values(by="total_sales_volume", ascending=False)

plt.figure(figsize=(18, 6))

sns.barplot(data=df_category_sales, x="prod_category", y="total_sales_volume", palette="magma")

plt.title("Total Sales Volume Distribution by Product Category", fontsize=14)
plt.xlabel("Product Category", fontsize=12)
plt.ylabel("Total Sales Volume", fontsize=6)
plt.xticks(rotation=45, ha="right")  
plt.show()


In [ ]:
# Display By Total Transaction
df_category_transactions = df_analysis.groupby("prod_category")["total_transactions"].sum().reset_index()
df_category_transactions = df_category_transactions.sort_values(by="total_transactions", ascending=False)

plt.figure(figsize=(18, 6))

sns.barplot(data=df_category_transactions, x="prod_category", y="total_transactions", palette="coolwarm")

plt.title("Total Transactions Distribution by Product Category", fontsize=14)
plt.xlabel("Product Category", fontsize=12)
plt.ylabel("Total Transactions", fontsize=6)
plt.xticks(rotation=45, ha="right") 

plt.show()


In [ ]:
# Based On Number of Customers
df_category_customers = df_analysis.groupby("prod_category")["unique_customers"].sum().reset_index()
df_category_customers = df_category_customers.sort_values(by="unique_customers", ascending=False)
plt.figure(figsize=(18, 6))

sns.barplot(data=df_category_customers, x="prod_category", y="unique_customers", palette="Blues_r")

plt.title("Unique Customers Distribution by Product Category", fontsize=14)
plt.xlabel("Product Category", fontsize=12)
plt.ylabel("Unique Customers", fontsize=6)
plt.xticks(rotation=45, ha="right")  


plt.show()


# Q3. Which stores rank the highest in volumes, revenues, profits, transactions, customers, etc.?

#### 3.0 Check the first transactions time for stores 

In [ ]:
# Define the SQL query to get the first transaction date for each store
query_store_open_date = """
WITH store_first_transaction AS (
    SELECT
        store_id,
        MIN(CAST(trans_dt AS DATE)) AS first_transaction_date
    FROM df_transactions
    GROUP BY store_id
)
SELECT *
FROM store_first_transaction
ORDER BY first_transaction_date DESC;
"""
df_stores_od = con.execute(query_store_open_date).fetchdf()
print("Store open dates:")
print(df_stores_od)

### 3.1 Overall Store Performance

In [ ]:
query_store_perf = """
WITH all_stores AS (
    SELECT DISTINCT store_id 
    FROM df_transactions
),

store_performance AS (
    SELECT
        s.store_id,
        ROUND(COALESCE(SUM(t.sales_amt), 0), 2) AS total_revenue,
        ROUND(COALESCE(SUM(t.sales_qty), 0), 2) AS total_volume,
        COUNT(DISTINCT t.trans_id) AS total_transactions,
        COUNT(DISTINCT t.cust_id) AS unique_customers
    FROM all_stores s
    LEFT JOIN df_transactions t
    ON s.store_id = t.store_id
    GROUP BY s.store_id
),

ranked_stores AS (
    SELECT *,
        RANK() OVER (ORDER BY total_revenue DESC) AS revenue_rank,
        RANK() OVER (ORDER BY total_volume DESC) AS volume_rank,
        RANK() OVER (ORDER BY total_transactions DESC) AS transaction_rank,
        RANK() OVER (ORDER BY unique_customers DESC) AS customer_rank
    FROM store_performance
)

SELECT *
FROM ranked_stores
ORDER BY revenue_rank;
"""
df_store_perf = con.execute(query_store_perf).fetchdf()
print("All-time store ranking:")
display(df_store_perf)


In [ ]:

plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")
sns.boxplot(x=df_store_perf['total_revenue'])

plt.title("Distribution of Total Revenue per Store", fontsize=14)
plt.xlabel("Total Revenue", fontsize=12)

# Show plot
plt.show()


## 3.3 Understanding the trend of the store performance

In [ ]:
df_transactions['trans_dt'] = pd.to_datetime(df_transactions['trans_dt'])
df_transactions['year_month'] = df_transactions['trans_dt'].dt.to_period('M')

# Group by store and month to get monthly sums/counts
monthly_store = (
    df_transactions
    .groupby(['store_id', 'year_month'], as_index=False)
    .agg(
        total_revenue=('sales_amt', 'sum'),
        total_transactions=('trans_id', 'nunique'),
        unique_customers=('cust_id', 'nunique')
    )
)
# Calculate average transaction value per store, per month
monthly_store['avg_transaction_value'] = (
    monthly_store['total_revenue'] / monthly_store['total_transactions']
)

# Get overall monthly averages across all stores
monthly_averages = (
    monthly_store
    .groupby('year_month', as_index=False)
    .agg(
        avg_transaction_value=('avg_transaction_value', 'mean'),
        avg_revenue_per_store=('total_revenue', 'mean'),
        avg_transactions_per_store=('total_transactions', 'mean'),
        avg_customer_count=('unique_customers', 'mean')
    )
)

# Display the results
display(monthly_averages)


In [ ]:
monthly_averages['year_month'] = monthly_averages['year_month'].astype(str)

# Plot Revenue Trend Over Time
plt.figure(figsize=(12, 6))
plt.plot(monthly_averages['year_month'], monthly_averages['avg_revenue_per_store'], marker='o', linestyle='-', color='blue')
plt.xticks(rotation=45)
plt.xlabel("Year-Month")
plt.ylabel("Average Revenue Per Store")
plt.title("Trend of Average Revenue Per Store Over Time")
plt.grid(True)
plt.show()

###According to the Capitalone research data,the average retail return rate is 26.4% for eCommerce and 10.0% for in-store purchases. so stores that have higher than 10% return should be flagged as abnormal.
https://capitaloneshopping.com/research/average-retail-return-rate/

# Q4. Are there interesting groupings of customers, e.g., most valuable (buy everything at any price) or cherry-pickers (buy mostly on promotions), defined by certain categories (buy baby products or never buy milk), etc.?

In [ ]:
import pandas as pd
import numpy as np

### 4.1 Customer Spending Behavior

- Product Category Analysis
- Price Sensitivity Analysis

##### Product Category Analysis

In [ ]:
# Unique product categories per customer
customer_category_diversity = df_combined.groupby('cust_id')['prod_category'].nunique().reset_index()
customer_category_diversity.columns = ['cust_id', 'unique_categories']

# Unique products per customer
customer_product_diversity = df_combined.groupby('cust_id')['prod_id'].nunique().reset_index()
customer_product_diversity.columns = ['cust_id', 'unique_products']

# Category focused customers versus broad shoppers
category_counts = df_combined.groupby(['cust_id', 'prod_category']).size().reset_index(name='category_purchase_count')
customer_total_purchases = df_combined.groupby('cust_id').size().reset_index(name='total_purchases')
category_concentration = category_counts.merge(customer_total_purchases, on='cust_id')
category_concentration['category_percentage'] = category_concentration['category_purchase_count'] / category_concentration['total_purchases']

# Top Category per customer
top_category_by_customer = category_concentration.sort_values(['cust_id', 'category_percentage'], ascending=[True, False])
top_category_by_customer = top_category_by_customer.drop_duplicates('cust_id')
top_category_by_customer = top_category_by_customer[['cust_id', 'prod_category', 'category_percentage']]
top_category_by_customer.columns = ['cust_id', 'top_category', 'top_category_percentage']

# Combining Purchase diversity metrics
purchase_diversity = customer_category_diversity.merge(
    customer_product_diversity, on='cust_id', how='left'
).merge(
    top_category_by_customer, on='cust_id', how='left'
)
# Fill NaN values
purchase_diversity = purchase_diversity.fillna(0)
display(purchase_diversity.head(20))

In [ ]:
# Prod category based customer segments
def categorize_diversity(top_percentage):
    if top_percentage >= 0.8:
        return "Top_category_shoppers"
    elif top_percentage >= 0.5:
        return "Category-Focused Shopper"
    elif top_percentage >= 0.3:
        return "Balanced Shopper"
    else:
        return "Diverse Shopper"

purchase_diversity['category_segment'] = purchase_diversity['top_category_percentage'].apply(categorize_diversity)

# Counting customers in each segment
df_shoppers = purchase_diversity['category_segment'].value_counts()
display(df_shoppers)

In [ ]:
# Average number of categories by segment
category_by_segment = purchase_diversity.groupby('category_segment')['unique_categories'].mean()
display(category_by_segment)

##### Price Sensitivity Analysis

In [ ]:
# Average price paid per product by each customer: How much each customer spends per product on average.
customer_price = stratified_sample_df.groupby('cust_id')['sales_amt'].mean().reset_index()
customer_price.columns = ['cust_id', 'avg_price_paid']

# Calculate overall average price for each product. This sets a baseline "market price" for each product.
product_avg_price = stratified_sample_df.groupby('prod_id')['sales_amt'].mean().reset_index()
product_avg_price.columns = ['prod_id', 'product_avg_price']

# Join average product price to transaction data
price_comparison = stratified_sample_df.merge(product_avg_price, on='prod_id')

# Price ratio for each transaction (customer price / average price). 
# Relative to the average price of the product, how much does a customer spend.
# >1.0 = less price-sensitive customer, 1.0 = avg prod price and <1.0 = price-sensitive cusotmer
price_comparison['price_ratio'] = price_comparison['sales_amt'] / price_comparison['product_avg_price']

# Average price ratio by customer
customer_price_sensitivity = price_comparison.groupby('cust_id')['price_ratio'].mean().reset_index()
customer_price_sensitivity.columns = ['cust_id', 'price_sensitivity']

# Q5. Other than product categories and sub-categories, are there other product groupings, e.g., Key Value Items (KVI) and Key Value Categories (KVC), traffic drivers, always promoted versus seldom/never promoted, etc.?

## 5.1 Analyze Product Groupings --- Key Value Items (KVI) & Key Value Categories (KVC)


In [ ]:
# Aggregate sales quantity by product subcategory
top_kvi_products = df_combined.groupby("prod_subcategory")["sales_qty"].sum().nlargest(10)

# Plot Key Value Items (KVI) by subcategory
plt.figure(figsize=(12, 6))
sns.barplot(x=top_kvi_products.index, y=top_kvi_products.values, palette="coolwarm")
plt.xticks(rotation=45, ha="right")
plt.title("Top 10 Key Value Product Subcategories")
plt.xlabel("Product Subcategory")
plt.ylabel("Total Sales Quantity")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


## 5.2 Analyze Product Groupings --- Traffic Drivers (Products Frequently Bought Alone)

In [ ]:
# Aggregate transaction counts by product subcategory
traffic_drivers = df_combined[df_combined.duplicated(subset=["trans_id"], keep=False) == False]
top_traffic_drivers = traffic_drivers.groupby("prod_subcategory")["prod_id"].count().sort_values(ascending=False).head(10)

# Plot Traffic Driver Products by Subcategory
plt.figure(figsize=(12, 6))
sns.barplot(x=top_traffic_drivers.index, y=top_traffic_drivers.values, palette="Blues_r")
plt.xticks(rotation=45, ha="right")
plt.title("Top 10 Traffic Driver Product Subcategories")
plt.xlabel("Product Subcategory")
plt.ylabel("Total Transactions")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


## 5.3 Analyze Product Groupings --- Always Promoted vs. Seldom/Never Promoted Items

In [ ]:
# Aggregate promotion counts by product subcategory
promoted_products = df_combined[df_combined["sales_amt"] < df_combined["sales_amt"].mean()]
promoted_counts = promoted_products.groupby("prod_subcategory")["prod_id"].count().sort_values(ascending=False).head(10)

# Plot Promoted Items with correct x and y axes
plt.figure(figsize=(12, 6))
sns.barplot(x=promoted_counts.index, y=promoted_counts.values, palette="Greens_r")
plt.xticks(rotation=45, ha="right")
plt.title("Top 10 Frequently Promoted Product Subcategories")
plt.xlabel("Product Subcategory")
plt.ylabel("Total Number of Promotions")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


## 5.4 Analyze Product Groupings --- Loss Leaders vs. High-Margin Products

In [ ]:
# Identify key drivers for most transactions (most purchased products, most revenue-generating stores)
query_key_drivers = """
SELECT prod_id, COUNT(trans_id) AS transaction_count, SUM(sales_amt) AS total_revenue
FROM `msba-emory.isom676_machine_learning.transactions`
GROUP BY prod_id
ORDER BY transaction_count DESC
LIMIT 10
"""

# Compare product prices over time (current vs. previous)
query_price_comparison = """
SELECT prod_id, trans_dt, sales_amt / sales_qty AS unit_price
FROM `msba-emory.isom676_machine_learning.transactions`
WHERE sales_qty > 0 AND trans_dt BETWEEN '2019-01-01' AND '2020-12-31'
ORDER BY prod_id, trans_dt
"""

# Monthly transaction trends for 2020
query_monthly_trends = """
SELECT EXTRACT(MONTH FROM trans_dt) AS month, COUNT(trans_id) AS transaction_count, SUM(sales_amt) AS total_revenue
FROM `msba-emory.isom676_machine_learning.transactions`
WHERE trans_dt BETWEEN '2020-01-01' AND '2020-12-31'
GROUP BY month
ORDER BY month
"""

# Check the effect of promotions
query_promotions_effect = """
SELECT prod_id, COUNT(trans_id) AS total_transactions,
       SUM(sales_amt) AS total_revenue,
       AVG(sales_amt / sales_qty) AS avg_price
FROM `msba-emory.isom676_machine_learning.transactions`
WHERE sales_amt < 0  -- Assuming negative sales_amt indicates promotions/discounts
GROUP BY prod_id
ORDER BY total_transactions DESC
LIMIT 10
"""

In [ ]:
# Executing the queries
query_results = {}
queries = {
    "Key Drivers": query_key_drivers,
    "Price Comparison": query_price_comparison,
    "Monthly Trends": query_monthly_trends,
    "Promotions Effect": query_promotions_effect
}

## 5.5 Prices between 2020 and 2019

In [ ]:
# Define the query for DuckDB
query_price_comparison = """
SELECT 
    prod_id,
    EXTRACT(YEAR FROM trans_dt) AS year,
    AVG(sales_amt / NULLIF(sales_qty, 0)) AS avg_unit_price
FROM read_parquet('C:/Users/Jiyuan Xin/Downloads/ACSEData/ACSEData/transactions.parquet')
WHERE trans_dt BETWEEN '2019-01-01' AND '2020-12-31'
GROUP BY prod_id, year
ORDER BY prod_id, year
"""

# Execute the query using DuckDB
df_price_comparison = con.execute(query_price_comparison).fetchdf()

# Pivot table to compare price changes between 2019 and 2020
df_price_pivot = df_price_comparison.pivot(index="prod_id", columns="year", values="avg_unit_price").reset_index()
df_price_pivot.columns = ["Product ID", "2019 Avg Price", "2020 Avg Price"]

# Calculate price difference and percentage change
df_price_pivot["Price Change"] = df_price_pivot["2020 Avg Price"] - df_price_pivot["2019 Avg Price"]
df_price_pivot["% Change"] = (df_price_pivot["Price Change"] / df_price_pivot["2019 Avg Price"]) * 100

# Display the results
print(df_price_pivot)

Monthly Level for 2020 (Most recent year)

In [ ]:
# Query to get monthly transaction trends for 2020
query_monthly_trends = """
SELECT
    EXTRACT(MONTH FROM trans_dt) AS month,
    COUNT(trans_id) AS transaction_count,
    SUM(sales_amt) AS total_revenue,
    AVG(sales_amt / NULLIF(sales_qty, 0)) AS avg_unit_price
FROM read_parquet('C:/Users/Jiyuan Xin/Downloads/ACSEData/ACSEData/transactions.parquet')
WHERE trans_dt BETWEEN '2020-01-01' AND '2020-12-31'
GROUP BY month
ORDER BY month
"""

# Execute the query using DuckDB
df_monthly_trends = duckdb.query(query_monthly_trends).to_df()

# Rename the month column for better readability
df_monthly_trends.rename(columns={"month": "Month", "transaction_count": "Total Transactions", "total_revenue": "Total Revenue", "avg_unit_price": "Avg Unit Price"}, inplace=True)

# Display the results
print(df_monthly_trends)  # Print the DataFrame to console


## 5.6 Always promoted Vs. Seldom/Never Promoted

In [ ]:
# Query to analyze always promoted vs seldom/never promoted products in 2020
query_promotion_analysis = """
WITH PromotionData AS (
    SELECT
        prod_id,
        COUNT(CASE WHEN sales_amt < 0 THEN trans_id END) AS promo_transactions,
        COUNT(trans_id) AS total_transactions,
        SUM(CASE WHEN sales_amt < 0 THEN sales_amt ELSE 0 END) AS promo_sales,
        SUM(sales_amt) AS total_sales
    FROM read_parquet('C:/Users/Jiyuan Xin/Downloads/ACSEData/ACSEData/transactions.parquet')
    WHERE trans_dt BETWEEN '2020-01-01' AND '2020-12-31'
    GROUP BY prod_id
)

SELECT
    prod_id,
    total_transactions,
    promo_transactions,
    (promo_transactions * 1.0 / total_transactions) AS promo_ratio,
    promo_sales,
    total_sales
FROM PromotionData
ORDER BY promo_ratio DESC
"""

# Execute the query using DuckDB
df_promotion_analysis = duckdb.query(query_promotion_analysis).to_df()

# Categorize products into "Always Promoted" vs. "Seldom/Never Promoted"
df_promotion_analysis["Promotion Category"] = df_promotion_analysis["promo_ratio"].apply(
    lambda x: "Always Promoted" if x > 0.75 else "Seldom/Never Promoted"
)

# Display the results
print(df_promotion_analysis)  # Print the DataFrame to console


In [ ]:
import matplotlib.pyplot as plt

# Sample data for visualization (since I cannot run BigQuery here)
sample_data = {
    "Promotion Category": ["Always Promoted", "Seldom/Never Promoted"],
    "Number of Products": [250, 750]  # Example counts
}

# Create a bar chart to show the number of products in each category
plt.figure(figsize=(8, 5))
plt.bar(sample_data["Promotion Category"], sample_data["Number of Products"], alpha=0.7)
plt.xlabel("Promotion Category")
plt.ylabel("Number of Products")
plt.title("Comparison of Always Promoted vs. Seldom/Never Promoted Products (2020)")
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Display the chart
plt.show()


In [ ]:
!pip install ace_tools

In [ ]:
# Define the SQL query to compute yearly discount rates for 2017-2020
query_discount_rate = """
WITH YearlyDiscount AS (
    SELECT
        EXTRACT(YEAR FROM trans_dt) AS year,
        SUM(CASE WHEN sales_amt < 0 THEN sales_amt ELSE 0 END) / SUM(ABS(sales_amt)) AS discount_rate
    FROM read_parquet('C:/Users/Jiyuan Xin/Downloads/ACSEData/ACSEData/transactions.parquet')
    WHERE EXTRACT(YEAR FROM trans_dt) BETWEEN 2017 AND 2020
    GROUP BY year
)

SELECT * FROM YearlyDiscount ORDER BY year;
"""

# Execute the query using DuckDB
df_discount_rate = duckdb.query(query_discount_rate).to_df()

# Display the results
display(df_discount_rate)  # Display the DataFrame

# Visualization of discount trends over the years
plt.figure(figsize=(8, 5))
plt.plot(df_discount_rate["year"], df_discount_rate["discount_rate"] * 100, marker="o", linestyle="-")
plt.xlabel("Year")
plt.ylabel("Discount Rate (%)")
plt.title("Yearly Discount Rates (2017-2020)")
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
# Query to get total sales quantity per product from 2017 to 2020
transactions_query = f"""
SELECT 
    prod_id, 
    SUM(sales_qty) AS total_sales
FROM read_parquet('C:/Users/Jiyuan Xin/Downloads/ACSEData/ACSEData/transactions.parquet')
WHERE trans_dt BETWEEN '2017-01-01' AND '2020-12-31'
GROUP BY prod_id
ORDER BY total_sales DESC
LIMIT 10
"""
top_promoted_products = con.execute(transactions_query).fetchdf()

# Load product descriptions from the products table
products_query = f"""
SELECT 
    prod_id, 
    prod_desc
FROM read_parquet('{products_path}')
"""
products_df = con.execute(products_query).fetchdf()

# Merge product descriptions
top_promoted_products = top_promoted_products.merge(products_df, on="prod_id", how="left")

# Display the result
print(top_promoted_products)


In [ ]:
display(top_10_promo_products)

In [ ]:

# Define file paths
transactions_path = "C:/Users/Jiyuan Xin/Downloads/ACSEData/ACSEData/transactions.parquet"

# Connect to DuckDB
con = duckdb.connect(database=':memory:')

# Corrected SQL Query
query_top_promo_percentage = f"""
WITH ProductYearlyAvg AS (
    -- Calculate the average yearly price per product
    SELECT
        prod_id,
        strftime('%Y', trans_dt) AS year,
        AVG(CASE WHEN sales_qty > 0 THEN sales_amt / NULLIF(sales_qty, 0) ELSE NULL END) AS avg_price
    FROM read_parquet('{transactions_path}')
    WHERE sales_qty > 0
    GROUP BY prod_id, strftime('%Y', trans_dt)  -- ✅ Group by the correct expression
),
PromoTransactions AS (
    -- Count total transactions and promotional transactions where price < yearly avg
    SELECT
        t.prod_id,
        strftime('%Y', t.trans_dt) AS year,
        COUNT(t.trans_id) AS total_transactions,
        COUNT(CASE WHEN (t.sales_amt / NULLIF(t.sales_qty, 0)) < p.avg_price THEN t.trans_id END) AS total_promo_transactions
    FROM read_parquet('{transactions_path}') t
    JOIN ProductYearlyAvg p
        ON t.prod_id = p.prod_id AND strftime('%Y', t.trans_dt) = p.year  -- ✅ Corrected column reference
    WHERE t.sales_qty > 0
    GROUP BY t.prod_id, strftime('%Y', t.trans_dt)  -- ✅ Group by original expression
)
-- Aggregate over four years and compute promo percentage
SELECT
    prod_id,
    SUM(total_promo_transactions) AS total_promo_transactions,
    SUM(total_transactions) AS total_transactions,
    (SUM(total_promo_transactions) * 1.0 / NULLIF(SUM(total_transactions), 0)) * 100 AS promo_percentage
FROM PromoTransactions
GROUP BY prod_id
ORDER BY promo_percentage DESC
LIMIT 10;
"""

# Execute the query and fetch results
top_promo_products_df = con.execute(query_top_promo_percentage).fetchdf()

from IPython.display import display

display(top_promo_products_df)




In [ ]:
# Define file paths
transactions_path = "C:/Users/Jiyuan Xin/Downloads/ACSEData/ACSEData/transactions.parquet"
products_path = "C:/Users/Jiyuan Xin/Downloads/ACSEData/ACSEData/products.parquet"

# Connect to DuckDB
con = duckdb.connect(database=':memory:')

# Corrected SQL Query
query_promo_distribution = f"""
WITH ProductYearlyAvg AS (
    -- Calculate the average yearly price for each product
    SELECT
        prod_id,
        strftime('%Y', trans_dt) AS year,
        AVG(CASE WHEN sales_qty > 0 THEN sales_amt / NULLIF(sales_qty, 0) ELSE NULL END) AS avg_price
    FROM read_parquet('{transactions_path}')
    WHERE sales_qty > 0
    GROUP BY prod_id, strftime('%Y', trans_dt)
),
PromoTransactions AS (
    -- Count total transactions and promotional transactions where price < yearly avg
    SELECT
        t.prod_id,
        strftime('%Y', t.trans_dt) AS year,
        COUNT(t.trans_id) AS total_transactions,
        COUNT(CASE WHEN (t.sales_amt / NULLIF(t.sales_qty, 0)) < p.avg_price THEN t.trans_id END) AS promo_transactions
    FROM read_parquet('{transactions_path}') t
    JOIN ProductYearlyAvg p
        ON t.prod_id = p.prod_id AND strftime('%Y', t.trans_dt) = p.year
    WHERE t.sales_qty > 0
    GROUP BY t.prod_id, strftime('%Y', t.trans_dt)
)
-- Aggregate by product category and subcategory
SELECT
    pr.prod_category,
    pr.prod_subcategory,
    SUM(pt.promo_transactions) AS total_promo_transactions,
    SUM(pt.total_transactions) AS total_transactions,
    (SUM(pt.promo_transactions) * 1.0 / NULLIF(SUM(pt.total_transactions), 0)) * 100 AS promo_percentage
FROM PromoTransactions pt
JOIN read_parquet('{products_path}') pr ON pt.prod_id = pr.prod_id
GROUP BY pr.prod_category, pr.prod_subcategory
ORDER BY total_promo_transactions DESC;
"""

# Execute the query and fetch results
df_promo_distribution = con.execute(query_promo_distribution).fetchdf()

# Display the top 10 rows
from IPython.display import display

display(top_promo_products_df)

# Sort data by Total Promo Transactions
df_sorted_promo_transactions = df_promo_distribution.sort_values(by="total_promo_transactions", ascending=False)

# First plot: Total Promo Transactions by Product Category (sorted)
plt.figure(figsize=(18, 6))
sns.barplot(data=df_sorted_promo_transactions, x="prod_category", y="total_promo_transactions")
plt.xticks(rotation=45, ha="right", fontsize=6)
plt.xlabel("Product Category")
plt.ylabel("Total Promo Transactions")
plt.title("Total Promo Transactions by Product Category (Sorted)")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

# Sort data by Promotion Percentage
df_sorted_promo_percentage = df_promo_distribution.sort_values(by="promo_percentage", ascending=False)

# Second plot: Promotion Percentage by Product Category (sorted)
plt.figure(figsize=(18, 6))
sns.barplot(data=df_sorted_promo_percentage, x="prod_category", y="promo_percentage")
plt.xticks(rotation=45, ha="right", fontsize=6)
plt.xlabel("Product Category")
plt.ylabel("Promotion Percentage (%)")
plt.title("Promotion Percentage by Product Category (Sorted)")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

# Q6. Are there natural groupings of stores, e.g., stores frequented by cherry-pickers versus stores visited by most loyal customers?## 6.1 Group by products specialization

## 6.1 Group by Store Revenue  (high, low, mid performance stores)

In [ ]:
query_store_revenue = """
SELECT
    store_id,
    ROUND(SUM(sales_amt), 2) AS total_revenue,
    CASE
        WHEN ROUND(SUM(sales_amt), 2)  > 8000000 THEN 'High-performance'
        WHEN ROUND(SUM(sales_amt), 2)  < 400000 THEN 'Low-performance'
        ELSE 'Medium-performance'
    END AS store_performance
FROM df_transactions
GROUP BY store_id;
"""
df_store_rev = con.execute(query_store_revenue).fetchdf()
print("Stores grouped by traffic level:")
display(df_store_rev)



In [ ]:
# Calculate the count of stores per performance category
performance_counts = df_store_rev['store_performance'].value_counts().reset_index()
performance_counts.columns = ['store_performance', 'count']

# Calculate the percentage for each performance category
performance_counts['percentage'] = (performance_counts['count'] / len(df_store_rev)) * 100

print("Store Performance Distribution (Count and Percentage):")
display(performance_counts)


### 6.1.1 Checking Store traffics

In [ ]:
query_store_traffic = """
SELECT
    store_id,
    COUNT(DISTINCT trans_id) AS total_transactions,
    COUNT(DISTINCT cust_id) AS unique_customers,
    CASE
        WHEN COUNT(DISTINCT trans_id) > 1000000 THEN 'High-Traffic'
        WHEN COUNT(DISTINCT trans_id) < 30000 THEN 'Low-Traffic'
        ELSE 'Medium-Traffic'
    END AS store_traffic_category
FROM df_transactions
GROUP BY store_id;
"""
df_store_traffic = con.execute(query_store_traffic).fetchdf()
print("Stores grouped by traffic level:")
display(df_store_traffic)


### 6.1.2 Checking Low performance store 

In [ ]:
query_low_perf_products = """
WITH low_perf_stores AS (
    SELECT store_id
    FROM df_store_rev
    WHERE total_revenue < 400000  -- Low-performing stores threshold
),

low_perf_product_sales AS (
    SELECT 
        t.store_id,
        p.prod_category,
        p.prod_subcategory,
        SUM(t.sales_amt) AS total_sales,
        COUNT(DISTINCT t.trans_id) AS total_transactions,
        COUNT(DISTINCT t.cust_id) AS unique_customers
    FROM df_transactions t
    JOIN df_products p
    ON t.prod_id = p.prod_id
    WHERE t.store_id IN (SELECT store_id FROM low_perf_stores)
    GROUP BY t.store_id, p.prod_category, p.prod_subcategory
),

category_summary AS (
    SELECT 
        prod_category,
        prod_subcategory,
        SUM(total_sales) AS total_sales,
        SUM(total_transactions) AS total_transactions,
        SUM(unique_customers) AS total_customers,
        COUNT(DISTINCT store_id) AS num_stores
    FROM low_perf_product_sales
    GROUP BY prod_category, prod_subcategory
    ORDER BY total_sales DESC
)

SELECT * FROM category_summary;
"""

# Execute the query
df_low_perf_categories = con.execute(query_low_perf_products).fetchdf()
display("Low-Performing Store Product Categories",df_low_perf_categories)


In [ ]:
query_high_perf_products = """
WITH high_perf_stores AS (
    SELECT store_id
    FROM df_store_rev
    WHERE total_revenue > 8000000  -- Low-performing stores threshold
),

high_perf_product_sales AS (
    SELECT 
        t.store_id,
        p.prod_category,
        p.prod_subcategory,
        SUM(t.sales_amt) AS total_sales,
        COUNT(DISTINCT t.trans_id) AS total_transactions,
        COUNT(DISTINCT t.cust_id) AS unique_customers
    FROM df_transactions t
    JOIN df_products p
    ON t.prod_id = p.prod_id
    WHERE t.store_id IN (SELECT store_id FROM high_perf_stores)
    GROUP BY t.store_id, p.prod_category, p.prod_subcategory
),

category_summary AS (
    SELECT 
        prod_category,
        prod_subcategory,
        SUM(total_sales) AS total_sales,
        SUM(total_transactions) AS total_transactions,
        SUM(unique_customers) AS total_customers,
        COUNT(DISTINCT store_id) AS num_stores
    FROM high_perf_product_sales
    GROUP BY prod_category, prod_subcategory
    ORDER BY total_sales DESC
)

SELECT * FROM category_summary
LIMIT 10;
"""

# Execute the query
df_high_perf_categories = con.execute(query_high_perf_products).fetchdf()
display("high-Performing Store Product Categories",df_high_perf_categories)


### 6.1.3 Understanding store 8540

In [ ]:
query_store_8540 = """
SELECT *
FROM df_combined
WHERE store_id = 8540
ORDER BY trans_dt;
"""

df_store_8540 = con.execute(query_store_8540).fetchdf()
print("Transactions for Store 8540:")
display(df_store_8540)


### 6.1.4 Group by products specialization

In [ ]:
query_store_prod = """
WITH store_product_mix AS (
    SELECT 
        t.store_id, 
        p.prod_category,  
        SUM(t.sales_amt) AS category_revenue
    FROM df_transactions AS t
    JOIN df_products AS p
      ON t.prod_id = p.prod_id
    GROUP BY t.store_id, p.prod_category
),
store_total_revenue AS (
    SELECT 
        store_id, 
        SUM(category_revenue) AS total_store_revenue
    FROM store_product_mix
    GROUP BY store_id
),
dominant_category AS (
    SELECT 
        spm.store_id, 
        spm.prod_category,
        spm.category_revenue,
        str.total_store_revenue,
        (spm.category_revenue * 100.0 / str.total_store_revenue) AS revenue_percentage,
        RANK() OVER (PARTITION BY spm.store_id ORDER BY spm.category_revenue DESC) AS category_rank
    FROM store_product_mix spm
    JOIN store_total_revenue str
      ON spm.store_id = str.store_id
)
SELECT 
    store_id, 
    prod_category AS dominant_category, 
    category_revenue, 
    total_store_revenue, 
    revenue_percentage
FROM dominant_category
WHERE category_rank = 1;

"""
df_store_prod = con.execute(query_store_prod).fetchdf()
print("Stores grouped by dominant product category:")
display(df_store_prod)


### 6.1.5 Checking coupon usage percentage

In [ ]:
query_coupon_percentage_all_stores = """
WITH store_transactions AS (
    SELECT 
        store_id,
        trans_id,
        sales_amt,
        prod_category
    FROM df_combined
),

coupon_usage AS (
    SELECT 
        store_id,
        COUNT(trans_id) AS coupon_count,
        ROUND(SUM(sales_amt), 2) AS coupon_sales  -- Aggregate sales amount for coupon transactions
    FROM store_transactions
    WHERE LOWER(prod_category) LIKE '%coupon%'  -- Identify coupon-related transactions
    GROUP BY store_id
),

total_transactions AS (
    SELECT 
        store_id,
        COUNT(trans_id) AS total_count,
        ROUND(SUM(sales_amt), 2) AS total_sales  -- Aggregate total sales per store
    FROM store_transactions
    GROUP BY store_id
)

SELECT 
    t.store_id,
    t.total_count,
    t.total_sales,  -- Total sales amount per store
    COALESCE(c.coupon_count, 0) AS coupon_count, 
    COALESCE(c.coupon_sales, 0) AS coupon_sales,  -- Total sales amount from coupons
    CONCAT(ROUND((COALESCE(c.coupon_count, 0) * 100.0) / NULLIF(t.total_count, 0), 2), ' %') AS coupon_usage_percentage,
    CONCAT(ROUND((COALESCE(c.coupon_sales, 0) * 100.0) / NULLIF(t.total_sales, 0), 2), ' %') AS coupon_sales_percentage
FROM total_transactions t
LEFT JOIN coupon_usage c ON t.store_id = c.store_id
ORDER BY CAST(REPLACE(coupon_usage_percentage, ' %', '') AS FLOAT) DESC;

"""

df_coupon_stats_all_stores = con.execute(query_coupon_percentage_all_stores).fetchdf()

# Display the coupon usage percentage for all stores
print("Coupon Usage Percentage for All Stores:")
display(df_coupon_stats_all_stores)


## 6.2 Group buy Customer type (cherry pickers and loyal customers)

### 6.2.1 cherry pickers 
Any customer whose discounted sales meet or exceed that threshold (i.e., they fall in the top 20% for discounted sales) is considered a cherry picker.

In [ ]:
query_store_customer_cp = """
WITH customer_metrics AS (
    SELECT
        t.cust_id,
        SUM(CASE
            WHEN (t.sales_amt / NULLIF(t.sales_qty, 0)) < p.prod_uom_value
            THEN t.sales_qty
            ELSE 0
        END) AS discounted_sales
    FROM df_transactions t
    JOIN df_products p ON t.prod_id = p.prod_id
    GROUP BY t.cust_id
),
cherry_picker_threshold AS (
    SELECT
        approx_quantile(discounted_sales, 0.8) AS top_20_cherry
    FROM customer_metrics
),
cherry_picker_customers AS (
    SELECT DISTINCT cm.cust_id
    FROM customer_metrics cm
    JOIN cherry_picker_threshold cpt ON cm.discounted_sales >= cpt.top_20_cherry
),
store_metrics AS (
    SELECT
        t.store_id,
        COUNT(DISTINCT t.cust_id) AS unique_customers,
        COUNT(DISTINCT CASE WHEN cpc.cust_id IS NOT NULL THEN t.cust_id END) AS cherry_picker_customers,
        ROUND(COUNT(DISTINCT CASE WHEN cpc.cust_id IS NOT NULL THEN t.cust_id END)
              / NULLIF(COUNT(DISTINCT t.cust_id), 0) * 100, 2) AS cherry_picker_percentage
    FROM df_transactions t
    LEFT JOIN cherry_picker_customers cpc ON t.cust_id = cpc.cust_id
    GROUP BY t.store_id
)
SELECT
    store_id,
    unique_customers,
    cherry_picker_customers,
    cherry_picker_percentage,
    CASE
        WHEN cherry_picker_percentage > 50 THEN 'Cherry-Picker Dominant Store'
        ELSE 'Not a Cherry-Picker Dominant Store'
    END AS store_customer_category
FROM store_metrics
ORDER BY cherry_picker_percentage DESC;
"""

# Execute the query in DuckDB and fetch the result as a DataFrame
df_store_customer_cp = con.execute(query_store_customer_cp).fetchdf()

# Display the results
print("Stores grouped by cherry picker behavior:")
display(df_store_customer_cp)


### 6.2.2 Loyal Customer
Any customer whose id starts with 1# (membership identification)

In [ ]:
query_store_customer_retention = """
WITH customer_purchases AS (
    SELECT
        cust_id,
        store_id,
        EXTRACT(YEAR FROM trans_dt) AS year,
        COUNT(*) AS transaction_count, -- Count total transactions per customer per year
        CASE 
            WHEN CAST(cust_id AS VARCHAR) LIKE '1%' THEN 1  -- Membership ID pattern
            ELSE 0
        END AS is_member
    FROM df_transactions
    GROUP BY cust_id, store_id, year
),

multi_year_customers AS (
    SELECT
        cust_id,
        store_id,
        COUNT(DISTINCT year) AS active_years, -- Count distinct years customer made a purchase
        SUM(transaction_count) AS total_transactions, -- Total transactions across years
        MAX(is_member) AS is_member -- Ensure membership status is retained
    FROM customer_purchases
    GROUP BY cust_id, store_id
),

customer_retention AS (
    SELECT
        store_id,
        COUNT(DISTINCT cust_id) AS total_customers,
        COUNT(DISTINCT CASE WHEN is_member = 1 THEN cust_id END) AS total_members,
        COUNT(DISTINCT CASE WHEN is_member = 0 THEN cust_id END) AS total_non_members,

        -- Retained customers: Multi-year presence + at least 12 transactions per year
        COUNT(DISTINCT CASE 
            WHEN active_years >= 1 AND total_transactions / active_years >= 3 AND is_member = 1 
            THEN cust_id 
        END) AS retained_members,

        COUNT(DISTINCT CASE 
            WHEN active_years >= 1 AND total_transactions / active_years >= 3 AND is_member = 0 
            THEN cust_id 
        END) AS retained_non_members
    FROM multi_year_customers
    GROUP BY store_id
),

store_retention_analysis AS (
    SELECT
        store_id,
        ROUND((retained_members * 100.0) / NULLIF(total_members, 0), 2) AS member_retention_rate,
        ROUND((retained_non_members * 100.0) / NULLIF(total_non_members, 0), 2) AS non_member_retention_rate,
        CASE 
            WHEN (ROUND((retained_members * 100.0) / NULLIF(total_members, 0), 2) >= 40) 
              OR (ROUND((retained_non_members * 100.0) / NULLIF(total_non_members, 0), 2) >= 20) 
              THEN 'Loyal-Customer Dominant Store'
            ELSE 'Not a Loyal-Customer Dominant Store'
        END AS store_customer_category
    FROM customer_retention
)

SELECT * 
FROM store_retention_analysis
ORDER BY member_retention_rate DESC, non_member_retention_rate DESC;
"""

df_store_customer_retention = con.execute(query_store_customer_retention).fetchdf()

# Display results
display("Store Retention Analysis",df_store_customer_retention)


In [ ]:
plt.figure(figsize=(10, 6))

# Scatter plot
sns.scatterplot(x=df_store_customer_retention["member_retention_rate"], 
                y=df_store_customer_retention["non_member_retention_rate"], 
                hue=df_store_customer_retention["store_customer_category"], 
                palette={"Loyal-Customer Dominant Store": "green", "Not a Loyal-Customer Dominant Store": "red"},
                s=100, alpha=0.7)

# Labels and Title
plt.axhline(y=20, color='black', linestyle='dashed', label="20% Non-Member Retention Benchmark")
plt.axvline(x=40, color='black', linestyle='dashed', label="40% Member Retention Benchmark")

plt.xlabel("Member Retention Rate (%)")
plt.ylabel("Non-Member Retention Rate (%)")
plt.title("Store Retention: Membership vs. Non-Membership")
plt.legend()
plt.show()
